In [1]:
!pip install langgraph langchain langchain-openai google-auth-oauthlib google-api-python-client langchain-google-genai


In [2]:
import os
import base64
from typing import TypedDict, Annotated, List, Literal
from email.mime.text import MIMEText

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import MemorySaver

# Google API Imports
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build



In [3]:
import os
os.listdir('/content')

['.config', 'token.json', 'credentials.json', 'sample_data']

In [4]:

GMAIL_SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
def authorize_gmail():
    creds = None

    # Check if token already exists
    if os.path.isfile('token.json'):
        creds = Credentials.from_authorized_user_file(
            'token.json', GMAIL_SCOPES
        )

    # If no valid credentials, start manual OAuth
    if not creds or not creds.valid:
        oauth_flow = InstalledAppFlow.from_client_secrets_file(
            'credentials.json',
            scopes=GMAIL_SCOPES,
            redirect_uri='urn:ietf:wg:oauth:2.0:oob'
        )

        auth_link, _ = oauth_flow.authorization_url(
            access_type='offline',
            include_granted_scopes='true'
        )

        print("\n OPEN THIS LINK IN YOUR BROWSER:\n")
        print(auth_link)

        auth_code = input("\n Enter the authorization code here: ").strip()

        oauth_flow.fetch_token(code=auth_code)
        creds = oauth_flow.credentials

        # Save token
        with open('token.json', 'w') as f:
            f.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)
    service = get_gmail_service()

In [5]:
gmail_service = authorize_gmail()
print("Gmail authorization successful")


Gmail authorization successful


In [6]:
def read_inbox(service):
    results = service.users().messages().list(
        userId='me',
        labelIds=['INBOX'],
        maxResults=5
    ).execute()

    messages = results.get('messages', [])

    if not messages:
        print("No messages found.")
        return

    for msg in messages:
        msg_data = service.users().messages().get(
            userId='me',
            id=msg['id'],
            format='metadata',
            metadataHeaders=['From', 'Subject']
        ).execute()

        headers = msg_data['payload']['headers']
        email_data = {}

        for h in headers:
            email_data[h['name']] = h['value']

        print("\n📧 New Email")
        print("From:", email_data.get('From'))
        print("Subject:", email_data.get('Subject'))

In [7]:
service = authorize_gmail()
print("Gmail service ready")

Gmail service ready


In [8]:
read_inbox(service)


📧 New Email
From: Google <no-reply@accounts.google.com>
Subject: Security alert

📧 New Email
From: "Makenna + IBM Dev Day Team" <makenna@bemyapp.com>
Subject: One Week Until IBM Dev Day: AI Demystified

📧 New Email
From: Hack2skill <hello@noreply.hack2skill.com>
Subject: 3 Days Left: Secure Your Spot in the ₹40 Lakh AWS Hackathon!

📧 New Email
From: Lalitha Rachabathuni <invitations@linkedin.com>
Subject: I want to connect

📧 New Email
From: Pinterest <recommendations@explore.pinterest.com>
Subject: Coord Set, Hand Work Blouse and more ideas to search for


In [10]:
from openai import OpenAI

client = OpenAI(api_key="open_ai_key")
print("Available OpenAI models:")
for model in client.models.list():
    print(model.id)


Available OpenAI models:
gpt-3.5-turbo
gpt-5.2-codex
gpt-4o-mini-tts-2025-12-15
gpt-realtime-mini-2025-12-15
gpt-audio-mini-2025-12-15
chatgpt-image-latest
davinci-002
babbage-002
gpt-3.5-turbo-instruct
gpt-3.5-turbo-instruct-0914
dall-e-3
dall-e-2
gpt-3.5-turbo-1106
tts-1-hd
tts-1-1106
tts-1-hd-1106
text-embedding-3-small
text-embedding-3-large
gpt-3.5-turbo-0125
gpt-4o
gpt-4o-2024-05-13
gpt-4o-mini-2024-07-18
gpt-4o-mini
gpt-4o-2024-08-06
gpt-4o-audio-preview
omni-moderation-latest
omni-moderation-2024-09-26
gpt-4o-audio-preview-2024-12-17
gpt-4o-mini-audio-preview-2024-12-17
o1-2024-12-17
o1
gpt-4o-mini-audio-preview
o3-mini
o3-mini-2025-01-31
gpt-4o-2024-11-20
gpt-4o-search-preview-2025-03-11
gpt-4o-search-preview
gpt-4o-mini-search-preview-2025-03-11
gpt-4o-mini-search-preview
gpt-4o-transcribe
gpt-4o-mini-transcribe
gpt-4o-mini-tts
o3-2025-04-16
o4-mini-2025-04-16
o3
o4-mini
gpt-4.1-2025-04-14
gpt-4.1
gpt-4.1-mini-2025-04-14
gpt-4.1-mini
gpt-4.1-nano-2025-04-14
gpt-4.1-nano
gpt-i

In [57]:
!pip install -U google-api-python-client google-auth google-auth-oauthlib google-genai


In [36]:
import os
import time
import base64
from email.mime.text import MIMEText

from google import genai
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build




In [38]:
def debug(msg):
    print(f"[DEBUG] {msg}")


In [37]:
os.environ["GEMINI_API_KEY"] = "gemini_api_key"

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])


In [14]:
SCOPES = ['https://www.googleapis.com/auth/gmail.modify']
TOKEN_PATH = "token.json"
CREDS_PATH = "credentials.json"

def get_gmail_service():
    creds = None

    if os.path.exists(TOKEN_PATH):
        creds = Credentials.from_authorized_user_file(TOKEN_PATH, SCOPES)

    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                CREDS_PATH, SCOPES
            )
            creds = flow.run_console()

        with open(TOKEN_PATH, 'w') as token:
            token.write(creds.to_json())

    return build('gmail', 'v1', credentials=creds)


In [17]:
service = get_gmail_service()

profile = service.users().getProfile(userId="me").execute()
print("Logged in as:", profile["emailAddress"])



Logged in as: bhavyamahali@gmail.com


In [39]:
def fetch_recent_emails(service, limit=2):
    results = service.users().messages().list(
        userId="me", q="is:unread", maxResults=limit
    ).execute()
    return results.get("messages", [])



In [40]:
import base64

def extract_email_parts(msg):
    msg_data = service.users().messages().get(
        userId="me", id=msg["id"], format="full"
    ).execute()

    headers = msg_data["payload"]["headers"]
    subject = sender = ""

    for h in headers:
        if h["name"] == "Subject":
            subject = h["value"]
        if h["name"] == "From":
            sender = h["value"]

    def get_body(payload):
        if "data" in payload.get("body", {}):
            return base64.urlsafe_b64decode(
                payload["body"]["data"]
            ).decode("utf-8", errors="ignore")

        for part in payload.get("parts", []):
            if part["mimeType"] == "text/plain":
                return base64.urlsafe_b64decode(
                    part["body"]["data"]
                ).decode("utf-8", errors="ignore")
        return ""

    body = get_body(msg_data["payload"])
    return sender, subject, body


In [41]:
def quick_ignore(sender, subject):
    sender = sender.lower()
    subject = subject.lower()

    blocked = [
        "linkedin", "internshala", "unstop",
        "pinterest", "noreply", "no-reply"
    ]

    return any(b in sender or b in subject for b in blocked)


def is_meeting_mail(subject):
    keywords = [
        "meeting", "invite", "invitation",
        "schedule", "call", "discussion"
    ]
    subject = subject.lower()
    return any(k in subject for k in keywords)


In [42]:
def gemini_call(prompt):
    response = client.models.generate_content(
        model="models/gemini-2.5-flash",
        contents=prompt
    )
    return response.text.strip()


In [43]:
def create_gmail_draft(service, to, subject, body):
    msg = MIMEText(body)
    msg["to"] = to
    msg["subject"] = subject

    raw = base64.urlsafe_b64encode(msg.as_bytes()).decode()

    draft = service.users().drafts().create(
        userId="me",
        body={"message": {"raw": raw}}
    ).execute()

    print("✅ Draft created in Gmail")
    return draft["id"]


In [44]:
def interactive_reply_and_draft(service, sender, subject, body):
    feedback = ""

    while True:
        print("\n--- Generating Draft... ---\n")

        prompt = f"""
Write a polite, human-like email reply.

Rules:
- Professional tone
- Short and clear
- Include greeting and closing

Sender: {sender}
Subject: {subject}
Email:
{body}

Feedback:
{feedback if feedback else "None"}
"""

        try:
            draft_text = gemini_call(prompt)
        except Exception:
            print("❌ Gemini quota hit. Try later.")
            return

        print("DRAFT PREVIEW:\n")
        print(f"To: {sender}")
        print(f"Subject: Re: {subject}")
        print("Body:\n")
        print(draft_text)
        print("\n----------------------------------")

        choice = input(
            "Action (yes / no / type feedback): "
        ).strip().lower()

        if choice in ["yes", "y"]:
            create_gmail_draft(
                service,
                sender,
                f"Re: {subject}",
                draft_text
            )
            return

        elif choice in ["no", "n"]:
            print("❌ Draft skipped")
            return

        else:
            print("Refining draft based on feedback...")
            feedback += f"\nUser wants: {choice}"


In [47]:
messages = fetch_recent_emails(service, limit=2)
print("Total unread mails:", len(messages))

for i, msg in enumerate(messages, 1):
    print(f"\n======= EMAIL {i} =======")

    sender, subject, body = extract_email_parts(msg)

    if quick_ignore(sender, subject):
        print("Decision: IGNORE")
        continue

    if not is_meeting_mail(subject):
        print("Decision: IGNORE")
        continue

    print("Decision: RESPOND")

    interactive_reply_and_draft(
        service,
        sender,
        subject,
        body
    )

    time.sleep(10)


Total unread mails: 2

======= EMAIL 1 =======
Decision: RESPOND

--- Generating Draft... ---

❌ Gemini quota hit. Try later.

======= EMAIL 2 =======
Decision: IGNORE
